# Data Cleaning - POS_CASH_balance

# 1. Import Library dan Dataset

In [1]:
# Import library dasar untuk manipulasi dan analisis data
import pandas as pd
import numpy as np

# Import library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

# Memastikan Jupyter Notebook menampilkan semua kolom
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')

In [3]:
pos_cash_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


# 2. Exploratory Data Analysis

## 2.1 Info dan Statistik Dasar dari DataFrame

In [4]:
#Menampilkan ringkasan statistik dari kolom numerik:
pos_cash_balance.describe()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF
count,1.000136e+07,1.000136e+07,1.000136e+07,9.975287e+06,9.975271e+06,1.000136e+07,1.000136e+07
mean,1.903217e+06,2.784039e+05,-3.501259e+01,1.708965e+01,1.048384e+01,1.160693e+01,6.544684e-01
std,5.358465e+05,1.027637e+05,2.606657e+01,1.199506e+01,1.110906e+01,1.327140e+02,3.276249e+01
min,1.000001e+06,1.000010e+05,-9.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.434405e+06,1.895500e+05,-5.400000e+01,1.000000e+01,3.000000e+00,0.000000e+00,0.000000e+00
50%,1.896565e+06,2.786540e+05,-2.800000e+01,1.200000e+01,7.000000e+00,0.000000e+00,0.000000e+00
75%,2.368963e+06,3.674290e+05,-1.300000e+01,2.400000e+01,1.400000e+01,0.000000e+00,0.000000e+00
max,2.843499e+06,4.562550e+05,-1.000000e+00,9.200000e+01,8.500000e+01,4.231000e+03,3.595000e+03


In [5]:
#Menampilkan ringkasan statistik dari kolom kategorikal:
pos_cash_balance.describe(include=['object'])

,NAME_CONTRACT_STATUS
count,10001358
unique,9
top,Active
freq,9151119


## 2.2 Checking Duplicate Values

In [6]:
pos_cash_balance.duplicated().sum()

0

Tidak terdapat duplicates value pada dataset "pos_cash_balance"

## 2.3 Checking Missing Values

In [7]:
pos_cash_balance.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [8]:
# Menghitung jumlah nilai hilang di setiap kolom
missing_values = pos_cash_balance.isnull().sum()

# Menghitung persentase nilai hilang di setiap kolom
missing_percentage = (missing_values / len(pos_cash_balance)) * 100

# Membuat DataFrame untuk menampilkan kolom dengan nilai hilang dan persentasenya
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Menampilkan kolom yang memiliki nilai hilang (nilai hilang > 0)
missing_data = missing_data[missing_data['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)

missing_data

,Missing Values,Percentage
CNT_INSTALMENT_FUTURE,26087,0.260835
CNT_INSTALMENT,26071,0.260675


Karena persentase missing value hanya 0.2% maka missing value diisi dengan nilai 0.

In [10]:
#Menyeleksi baris dalam DF 'pos_cash_balance', dimana nilai kolom  'CNT_INSTALMENT' adalah NaN (tidak ada).
pos_cash_balance.loc[pos_cash_balance.CNT_INSTALMENT.isna()]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
709,1889585,403422,-6,NaN,NaN,Signed,0,0
759,1618886,382448,-2,NaN,NaN,Signed,0,0
1887,2288203,429576,-6,NaN,NaN,Signed,0,0
1899,2110224,167171,-6,NaN,NaN,Signed,0,0
1910,2031967,235187,-5,NaN,NaN,Signed,0,0
...,...,...,...,...,...,...,...,...
9998668,1770932,441177,-10,NaN,NaN,Signed,0,0
9998696,1770932,441177,-11,NaN,NaN,Signed,0,0
9999114,1770932,441177,-8,NaN,NaN,Signed,0,0
9999116,1770932,441177,-9,NaN,NaN,Signed,0,0


In [12]:
pos_cash_balance_cleaned = pos_cash_balance.fillna(0)
pos_cash_balance_cleaned

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [13]:
csv_file_path = 'C:/Users/Inne Andarini/Downloads/pos_cash_balance_cleaned.csv'
pos_cash_balance_cleaned.to_csv(csv_file_path, index=False)